In [122]:
import numpy as np
import pickle
import sklearn.feature_extraction.text as text_a
import re
from collections import Counter, defaultdict
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.decomposition import PCA
from tqdm import tqdm
from scipy import spatial
from nltk.corpus import stopwords
import json
from nltk.cluster.kmeans import KMeansClusterer
from scipy.spatial.distance import cosine
import random
import pickle
nltk.data.path.append("/home")

lemmatizer = WordNetLemmatizer()

In [123]:
# Все тексты проф. фейсбука
texts = pickle.load(open("answer.pck", "rb"))
# Разделы проф.
tuples = pickle.load(open("tuples (2).pck", "rb"))

# Почистим тексты
for i in range(len(texts)):
    texts[i] = texts[i][texts[i].replace("."," ").index("Responsibilities"):][16:-152]

In [124]:
# Все курсы
coursera = pickle.load(open("coursera.p", "rb"))

# Titles работ
jobs_topic_names = pickle.load(open("job_topics.pck", "rb"))

In [125]:
# Все тексты курсов
coursera_texts = [i["description"] for i in coursera]

# Titles курсов
coursera_topic_names = [i["title"] for i in coursera]

In [126]:
# Все топики
all_topics = jobs_topic_names + coursera_topic_names

In [127]:
def clear_text(text):
    """Очистка текста и начальная форма слов"""
    global lemmatizer
    
    tmp_text = ""
    
    text = text.lower().replace("\n", " ")
    
    for char in "!@#$%^&*()[]{};:,./<>?|`~-=_+●\\":
        text = text.replace(char, " ")
        
    text = re.sub('[^A-Za-z]+ ', '', text).split(" ")
    
    for word in text:
        tmp_text += lemmatizer.lemmatize(word, 'v') + " "
    
    return tmp_text

In [128]:
# почистим проф.
texts = list(map(clear_text, texts))

In [129]:
# почистим курсеру
coursera_texts = list(map(clear_text, coursera_texts))

In [130]:
k = list(map(clear_text, all_topics))

In [131]:
tmp_text = ""

for text in texts:
    tmp_text += text + " "

job_words = set(tmp_text.split(" "))

tmp_text = ""

for text in coursera_texts:
    tmp_text += text + " "

coursera_words = set(tmp_text.split(" "))

white_list = job_words & coursera_words

In [132]:
tmp = []

for text in tqdm(texts):
    tmp_text = ""
    for word in text.split(" "):
        if word not in set(stopwords.words('english')):
            tmp_text += word + " "

    tmp.append(tmp_text)

texts = tmp

tmp = []

for text in tqdm(coursera_texts):
    tmp_text = ""
    for word in text.split(" "):
        if word not in set(stopwords.words('english')):
            tmp_text += word + " "

    tmp.append(tmp_text)

coursera_texts = tmp

100%|██████████| 1499/1499 [00:22<00:00, 66.46it/s]


In [133]:
all_texts = texts + coursera_texts

In [134]:
vectorizer = text_a.TfidfVectorizer()
dtm = vectorizer.fit_transform(k).toarray()

In [149]:
cluster = KMeansClusterer(20, spatial.distance.cosine)
cluster.cluster(dtm)

/anaconda3/lib/python3.5/site-packages/scipy/spatial/distance.py:326: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


In [136]:
cluster = pickle.load(open("cluster.pck","rb"))

In [150]:
clusters_to_titles = defaultdict(lambda: [])

for c,d in zip([cluster.classify(i) for i in dtm], jobs_topic_names+coursera_topic_names):
    clusters_to_titles[c].append(d)

/anaconda3/lib/python3.5/site-packages/scipy/spatial/distance.py:326: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


In [151]:
doctopic = [cluster.vector(i) for i in dtm]

In [152]:
def sevalg(user_vector,item_vector):
    tmp = []
    
    for n,m in zip(user_vector, item_vector):
        if n == 0:
            tmp.append(m)
        else:
            tmp.append(1)
    
    return np.array(tmp)

In [153]:
items_vectors = {}

for a,b in zip(jobs_topic_names + coursera_topic_names, dtm):
    state = b in coursera_texts
    items_vectors[a] = (b,state)

/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [223]:
def get_courses_for_users(user_vector, job_name):
    '''Основная функция, которая по вектору пользователя и названию работы возращает курсы'''
    item_vector = items_vectors[job_name][0]
    answer = []
    tmp = []
    for cluster_center in cluster._means:
        tmp.append(spatial.distance.cosine(cluster_center, item_vector))

    v = sevalg(user_vector, tmp)
    print(v)

    for vector in coursera_topic_names:
        tmp = []
        for cluster_center in cluster._means:
            #print(sum(items_vectors[vector][0]))
            tmp.append(spatial.distance.cosine(cluster_center,  items_vectors[vector][0]))

        try:
            print(sum(v), sum(tmp))
            answer.append(spatial.distance.euclidean(v, tmp))
        except ValueError as error:
            print(error)
            answer.append(10**5)
            continue

    d = Counter()
    for i, j in enumerate(answer):
        d[i] = j
    res = []
    for i, j in d.most_common()[::-1]:
        try:
            # Есть курсы названия которых содержут не читаемые символы, уберём их
            coursera_topic_names[i].encode('ascii')
            
            # Тут мы убераем все курсы на немецких, хрен каких языках, оставляем только красивые.
            if (' de ' not in coursera_topic_names[i].lower() and ' en ' not in coursera_topic_names[i].lower() and
                ' el ' not in coursera_topic_names[i].lower() and ' con ' not in coursera_topic_names[i].lower()):
                res.append(coursera_topic_names[i])
        except UnicodeEncodeError:
            continue
    return res
    real_answer = []
    
    for i in np.argsort(answer)[:10]:
        real_answer.append(coursera_topic_names[i])
    print(real_answer)
    return real_answer

In [224]:
coursera = pickle.load(open('coursera.p', 'rb'))

def get_meta(recommendations):
    meta = [x for x in coursera if x['title'] in recommendations[:25]]
#     # все из специализации для каждого в начало
#     for course in meta:
#         if 'specialization' in course:
#             for insert in list(filter(
#                     lambda x: 'specialization' in x and x['specialization'] == course['specialization'],
#                     coursera)):
#                 result.append(insert)
#     for course in meta:
#         if 'specialization' not in course:
#             result.append(course)
#     print(result)
    return json.dumps(meta)

In [225]:
tuples = pickle.load(open("tuples (2).pck", "rb"))

In [226]:
ALL_JOBS = []

for j in tuples:
    ALL_JOBS.append([tuples[j][0],tuples[j][1],j])

ALL_JOBS = json.dumps(ALL_JOBS)

In [227]:
from flask import Flask
app = Flask(__name__)

In [228]:
@app.route('/recommend/<vector>&<job>')
def recommend(vector, job):
    return get_meta(get_courses_for_users(list(map(int, vector.split(","))), job))

In [229]:
@app.route("/all_jobs")
def hello():
    return ALL_JOBS

In [ ]:
app.run(host="0.0.0.0", port=8080)

 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
54.74.90.107 - - [11/Dec/2016 13:39:04] "GET /all_jobs HTTP/1.1" 200 -
54.220.151.224 - - [11/Dec/2016 13:39:34] "GET /all_jobs HTTP/1.1" 200 -
54.220.151.224 - - [11/Dec/2016 13:39:36] "GET /all_jobs HTTP/1.1" 200 -
54.220.151.224 - - [11/Dec/2016 13:39:42] "GET /all_jobs HTTP/1.1" 200 -


[ 0.97589636  1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.        ]
19.9758963644 19.3807922754
19.9758963644 19.2171509398
19.9758963644 19.3693990839
19.9758963644 19.2881060489
19.9758963644 18.8317612216
19.9758963644 19.4196007199
19.9758963644 19.4067755785
19.9758963644 19.4415833726
19.9758963644 19.9829262778
19.9758963644 19.5680696054
19.9758963644 19.5639139175
19.9758963644 19.272588067
19.9758963644 19.9722727364
19.9758963644 18.8320796853
19.9758963644 19.457471472
19.9758963644 19.3246501593
19.9758963644 19.6597683511
19.9758963644 19.6898597351
19.9758963644 19.4986115519
19.9758963644 18.6135012235
19.9758963644 19.4266455388
19.9758963644 19.4189444868
19.9758963644 19.0129333597
19.9758963644 18.6726050275
19.9758963644 19.0202172007
19.9758963644 19.2726889329
19.9758963644 19.86044757
19.9758963644 19.545

/anaconda3/lib/python3.5/site-packages/scipy/spatial/distance.py:326: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


19.9758963644 19.480844591
19.9758963644 19.4878398783
19.9758963644 19.7200446317
19.9758963644 19.5994947927
19.9758963644 19.1814918625
19.9758963644 19.0964026844
19.9758963644 19.126353015
19.9758963644 19.1974410806
19.9758963644 19.8631288652
19.9758963644 18.9796251515
19.9758963644 19.4943531319
19.9758963644 19.3356705738
19.9758963644 19.9176040924
19.9758963644 19.5664963508
19.9758963644 19.6196536807
19.9758963644 18.6705588759
19.9758963644 19.5732354032
19.9758963644 18.9164237537
19.9758963644 19.4673982312
19.9758963644 19.5830227364
19.9758963644 19.9223948138
19.9758963644 19.1608666513
19.9758963644 19.3686834315
19.9758963644 19.5519313712
19.9758963644 18.9645592686
19.9758963644 19.3651736049
19.9758963644 19.4136208565
19.9758963644 19.9695605787
19.9758963644 19.0468523935
19.9758963644 19.6956089994
19.9758963644 19.1475068028
19.9758963644 18.654394932
19.9758963644 19.6444198936
19.9758963644 19.3844112261
19.9758963644 19.4164983697
19.9758963644 19.900385

54.220.151.224 - - [11/Dec/2016 13:39:46] "GET /recommend/0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0&Graphics%20Engineer,%20Building%208 HTTP/1.1" 200 -


19.9758963644 19.7294898604
19.9758963644 19.4869781367
19.9758963644 19.2598047482
19.9758963644 19.9675117136
19.9758963644 18.4018243471
19.9758963644 18.9142258343
19.9758963644 19.7505515291
19.9758963644 19.8561927977
19.9758963644 19.6243559266
19.9758963644 19.7812419761
19.9758963644 19.3459826879
19.9758963644 19.3211358889
19.9758963644 19.4808571479
19.9758963644 19.7251571628
19.9758963644 19.4929123343
19.9758963644 19.6949011699
19.9758963644 18.8926619231
19.9758963644 19.3459381012
19.9758963644 19.9222527435
19.9758963644 19.4482156739
19.9758963644 18.6488312536
19.9758963644 19.7865261698
19.9758963644 19.1616495651
19.9758963644 19.9340396851
19.9758963644 19.6499381344
19.9758963644 19.5404892407
19.9758963644 19.9016891365
19.9758963644 19.44417707
19.9758963644 19.9552131188
19.9758963644 19.480587936
19.9758963644 19.1176856158
19.9758963644 18.684373018
19.9758963644 19.1625258821
19.9758963644 18.4403960147
19.9758963644 19.7181961734
19.9758963644 18.6737345

54.220.151.224 - - [11/Dec/2016 13:40:13] "GET /all_jobs HTTP/1.1" 200 -
54.220.151.224 - - [11/Dec/2016 13:40:17] "GET /all_jobs HTTP/1.1" 200 -
54.220.151.224 - - [11/Dec/2016 13:40:22] "GET /all_jobs HTTP/1.1" 200 -


[ 0.98853958  0.85314702  0.99119316  1.          0.99013846  1.
  0.98473903  0.71841498  0.90433108  1.          0.99190956  0.98973902
  0.99089405  0.95079456  0.97935984  0.99102593  1.          0.99417661
  0.85574111  0.98710008]
19.1612440872 19.3807922754
19.1612440872 19.2171509398
19.1612440872 19.3693990839
19.1612440872 19.2881060489
19.1612440872 18.8317612216
19.1612440872 19.4196007199
19.1612440872 19.4067755785
19.1612440872 19.4415833726
19.1612440872 19.9829262778
19.1612440872 19.5680696054
19.1612440872 19.5639139175
19.1612440872 19.272588067
19.1612440872 19.9722727364
19.1612440872 18.8320796853
19.1612440872 19.457471472
19.1612440872 19.3246501593
19.1612440872 19.6597683511
19.1612440872 19.6898597351
19.1612440872 19.4986115519
19.1612440872 18.6135012235
19.1612440872 19.4266455388
19.1612440872 19.4189444868
19.1612440872 19.0129333597
19.1612440872 18.6726050275
19.1612440872 19.0202172007
19.1612440872 19.2726889329
19.1612440872 19.86044757
19.16124408

54.220.151.224 - - [11/Dec/2016 13:40:27] "GET /recommend/0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0&Marketing%20Research%20Manager,%20Building%208 HTTP/1.1" 200 -


19.1612440872 19.5133557617
19.1612440872 19.2178570631
19.1612440872 19.6082811649
19.1612440872 19.28333867
19.1612440872 19.1380324177
19.1612440872 19.3405720771
19.1612440872 19.1617529178
19.1612440872 19.8409036728
19.1612440872 19.9560523894
19.1612440872 19.4529589849
19.1612440872 19.5463736842
19.1612440872 19.0786930545
19.1612440872 19.4731270339
19.1612440872 18.7473956053
19.1612440872 19.6600552424
19.1612440872 19.4641396475
19.1612440872 19.4389476064
19.1612440872 19.496138013
19.1612440872 19.6339646881
19.1612440872 19.1790794028
19.1612440872 19.244059811
19.1612440872 19.3867250134
19.1612440872 19.7299351684
19.1612440872 19.5627551521
19.1612440872 19.4049643685
19.1612440872 19.5220114096
19.1612440872 19.0364726232
19.1612440872 18.6956274926
19.1612440872 19.9829262778
19.1612440872 19.1675174394
19.1612440872 19.2851842191
19.1612440872 19.0712111232
19.1612440872 19.1660078423
19.1612440872 19.9615635894
19.1612440872 19.2264233497
19.1612440872 19.1460944